# Data Analysis and Muon Lifetime

In this notebook we will discuss histograms, statistics, curve fitting, and analyzing simulated data from the muon lifetime experiment.



**First things first, make sure you save this notebook to your drive so you can save your changes!!!**

<div>
<img src="save_to_drive.png" width="250"/>
</div>

In [ ]:
# import libraries needed.  Run this block of code without modification.

%matplotlib notebook
%matplotlib inline

import numpy as np
import scipy.optimize as opt
import scipy.integrate as integrate
import scipy.stats as stats
import matplotlib.pyplot as plt
import inspect

### Exercise 1: Define a gaussian distribution

The equation for a gaussian distribution is:

$$ \Large f(x)=\frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}$$

Where $\mu$ is the mean and $\sigma$ is the width.  Define a python function called `gaussian` which accepts 3 arguments: `x`, `sigma` and `mean`.  The default value for `sigma` shoud be `1.0` and for mean `0.0`.  Default values are specified in the arugments to the function as `def func(arg1, arg2=1.0):`.

You will need `np.sqrt`, `np.exp`, `np.power`, `np.pi` for square roots, exponentials, raising a number to a power, and using $\pi$.  Note that `np.pi` is a *member*, not a *function*, so you do not need the parentheses `()` when using `np.pi`.

Plot your gaussian function by defining an array of points using `np.linspace`.  Then use `plt.plot(x,y)` to plot that array and the gaussian function.  Make sure that when you change `sigma` and `mean` you get the expected result.

### Exercise 2: Define a Laplace distribution

A gaussian is not the only distribution which has a mean and width.  In fact, you may find the functional form of a gaussian distribution strange.  Why is $x$ squared in the exponent, for example?  Why is $\pi$ there, and why is it the square root of $\pi$?  It seems like a strange choice for a distribution!  We can also define the Laplace, or "double exponential" distribution, where just the absolute value of $x$ is in the exponent, not $x^2$:

$$ \Large f(x)=\frac{1}{\sqrt{2}\sigma}e^{\left(\frac{-\sqrt{2}|x-\mu|}{\sigma}\right)}$$

Define the Laplace distribution below.  Use the function `np.abs` to compute the absolute value.  Again, the arguments are `x`, `sigma`, and `mean`, and `sigma` should default to `1.0` and mean should default to `0.0`.

Similarly to the gaussian distribution, plot the laplace distribution and verify that changing the `sigma` and `mean` make sense.  Try plotting them both together with the same value of `sigma` and `mean`.  Use `plt.plot(x,y)`.

The distributions have some similarities and some differences!  They appear to have very roughly the same "width" when we set the value of `sigma` to be the same for both distributions.  However, the Laplace distribution is pointy and has more probability "mass" concentrated near the center while the gaussian distribution is smooth and more spread out.  Later in this notebook we will explore fitting a set of observed data to both distributions and seeing which one fits the data better.

### Exercise 3: Normalization of Probability Density Functions

One property of a **Probability Density Function** or PDF is that its integral is exactly 1.  You can think of this as adding up all the individual probabilities of every possible outcome and getting 100%.  Verify that the integral of these distributions is 1 regardless of the choice of `mean` and `sigma`.  To perform this integral numerically, perform the following steps:


1. Create a linearly spaced set of points `x` using `np.linspace`
2. Call the function, either `gaussian(x)` or `laplace(x)` over the linearly spaced set of points.
3. Use the built-in function `sum` to add together all of the values.
4. Multiply by d$x$: the spacing between points.  You can find it by typing: `dx=x[1]-x[0]`.  The function value is the "height" of the rectangle and d$x$ is the "width".
5. Try several different values for `sigma` and `mean`.  Make sure that if you change them you also appropriately change the range.  The range should be at least from `mean-4*sigma` to `mean+4*sigma`.
6. Optional challenge: write a new function which numerically integrates a user provided function over the bounds `a` to `b` using the "algorithm" defined here.  Now, you can easily reuse your function to integrate several PDFs.  Completing this challenge will require you to use a special new syntax: passing the argument `**kwargs` to the new function you write to perform your numerical integration.  If python is a challenge for you then you can feel free to skip this exercise.


### Fitting Histograms

The code below defines a function `plot_data_and_fit` which accepts a list or numpy array of observed data and creates a histogram and fits it to a function of your choice.

Read the code and try to understand the algorithm even if you don't understand each individual line of code.  Then, run the block of code with no modification.

In [ ]:
#
# DO NOT MODIFY THIS CODE BLOCK

# define a style dictionary for data-like plotting
data_style={'color':'k', 'marker':"o", 'linestyle':'None', 'label':'Observed Data'}

'''
The method to plot a histogram and fit it to an arbitrary function.

Arguments:
 - observed data: array-like object containing all of the individual observations
 - num_bins:      number of bins for the histogram.  Default=10
 - function:      callable object representing the function we will fit to the data.
                  Default= the gaussian function you defined earlier.
 - guess:         Initial guess of parameter values.  If no guess is provided then it is initialized to 1 for
                  each parameter
'''
def plot_data_and_fit(observed_data, num_bins=10, function=gaussian, guess=[], doPlot=True):
    # get information about the fitting function.  The independent variable name, fit parameters, function name
    x_axis_name   = inspect.getfullargspec(function).args[0]
    callable_args = inspect.getfullargspec(function).args[1:]
    callable_name = [i for i, j in globals().items() if j is function][0]
    if isinstance(guess, float) or isinstance(guess, int):
        guess = [guess]
    if len(guess) != len(callable_args):
        guess = [1.0]*len(callable_args) # list concatenation!

    # create a histogram and compute the error on each observation
    observed_counts, bin_edges = np.histogram(observed_data, bins=num_bins)
    bin_width   = (bin_edges[1:] - bin_edges[:-1])[0]
    bin_centers = bin_width/2 + bin_edges[:-1]
    observed_errors = np.sqrt(observed_counts)

    # define the chi-square test statistic.  Integrates the input function.  Automatically normalizes to N observations.
    def test_statistic(x):
        expected_counts = np.asarray([integrate.quad(function, a, b, args=tuple(x))[0] for a, b in zip(bin_edges[:-1], bin_edges[1:])])
        return stats.chisquare(observed_counts, expected_counts*sum(observed_counts)/sum(expected_counts), ddof=len(x)).statistic
    
    # minimize the chi square test statistic.  Get the best fit parameters.
    optimizedResult = opt.minimize(test_statistic, x0=guess)
    
    optimizedParameters = optimizedResult.x

    # use the hessian matrix and residual error to estimate the error bars on the best fit parameters.
    fitted_errors = np.sqrt( np.diag( optimizedResult.hess_inv * optimizedResult.fun))

    # print important information to the screen
    probability = 1-stats.chi2.cdf(optimizedResult.fun, float(len(bin_centers)-len(guess)))
    integral_factor = integrate.quad(function, bin_edges[0], bin_edges[-1], args=tuple(optimizedParameters))[0]
    if doPlot:
        print("Chi square: ", optimizedResult.fun)
        print("N dof: ", len(bin_centers)-len(guess))
        print("Chi square Probability: ", probability)

        for i in range(len(optimizedParameters)):
            print(callable_args[i], ":", optimizedParameters[i], "+/-", fitted_errors[i])

        # Plot the histogram of the observed data
        plt.errorbar(bin_centers, observed_counts, yerr=observed_errors, **data_style)

        # Plot the fitted function
        x = np.linspace(min(observed_data), max(observed_data), 100)
        plt.plot(x, len(observed_data)*bin_width*function(x, *optimizedParameters)/integral_factor, 'r', label=callable_name + ' Fit')

        # beautify plot
        plt.xscale('linear')
        plt.yscale('linear')
        plt.xlabel(x_axis_name)
        plt.ylabel('Entries')
        plt.title('Histogram and ' + callable_name + ' Fit')
        plt.legend()
        plt.show()
    
    # return output
    output = {'prob': probability, 
              'chi-sq': optimizedResult.fun,
              'NDF': len(bin_centers)-len(guess)
               }
    for i in range(len(callable_args)):
        output[callable_args[i]] = (optimizedParameters[i], fitted_errors[i])
    return output


### Exercise 4: Fit gaussian data

Generate some gaussian data with the function `np.random.normal(size=<N>)` where you can choose the number of points `<N>`.  Then, call the `plot_data_and_fit` function with your generated data.  By default it will use the gaussian function to fit this data.  You may want to define a reasonable value for the number of bins.  I suggest 1000 data points and 30 bins to start.

You should see a set of black data points with error bars representing the range of values that might be observed if you repeated the experiment.  The red curve is the best-fit gaussian.  It also prints out the best fit value of the `sigma` and `mean`, with errors.  The $\chi^2$ test statistic and corresponding probability are also printed.

Next, try fitting the same data with the `laplace` function.  Compare the probability printout.  Is it more likely that the `gaussian` or `laplace` distribution describes this data?

Many processes in nature are described by guassian distributions (It is the "central limit" and "maximum entropy" probability distribution. If you are curious what this means check out 3Blue1Brown's videos about gaussians on YouTube).

### Exercise 5: Goodness of Fit and $\chi^2$ Probability

If we pick the **right** probability distribution then as we collect more data the value of the $\chi^2$ probability should be more or less unchanged.  If we pick the **wrong** probability distribution to fit the data then as we collect more data it should become more and more clear that we made the wrong choice.  Therefore, the $\chi^2$ probability should decrease as we collect more data, if we made the wrong choice.  Write a `for` loop which on each iteration:

1. Creates a new set of gaussian data with the `size` parameter increasing.  The suggested sizes are already defined for you.
2. Fits the data to both a `gaussian` and a `laplace` distribution.  Save the dictionary returned by each call to `plot_data_and_fit`.  Use the `doPlot=False` argument in `plot_data_and_fit` so that it doesn't try to plot each individual fit.
3. Use the list `append` method to add each successive probability to the `gaus_probs` and `lapl_probs` lists.  You can access the probability in the returned dictionary like `dict["prob"]`.

After your `for` loop you will use `plt.plot` to plot `gaus_probs` and `lapl_probs` on the y-axis as a function of `sizes` on the x-axis.  You will also need to use `plt.xscale('log')` and `plt.yscale('log')` to make the axes have a logarithmic scale.  Try reading the above code that defines `plot_data_and_fit` to set the axis labels and make a legend for your plot.

Sometimes, depending on the generated data, the `plot_data_and_fit` function will complain and produce an error.  You can ignore this.

In [ ]:
sizes = [100, 200, 300, 400, 500, 800, 1000, 1500, 2000, 3000, 5000]
gaus_probs = []
lapl_probs = []

for s in sizes:
    # write a for loop that appends to the two lists

    
# draw a plot with the two lists on the y axis and sizes on the x axis.

About how many data points do we need to start to tell the difference between data distributed like a gaussian or a laplace distribution?

In particle physics we define "discovery" as a probability less than $3\times10^{-7}$.  How many observations would you need to "discover" that this data is not distributed like a laplace distribution?

### Exercise 6: Muon Lifetime

Now that we understand how to fit data and digest the results, we can try to fit the data from our muon lifetime experiment.

The muon decays to an electron and two neutrinos.  Particles that decay should have lifetimes distributed according to an exponential distribution

$$\Large f(t)=\frac{1}{\tau}e^{-t/\tau}$$

Where $t$ is the measured time and $\tau$ is the lifetime of the muon.

First, define a function `exponential` which accepts two arguments: `t` and `tau`.  It should return the function $f(t)$ defined mathematically above.  You can use the `np.exp` function to perform the exponential.

Run this code to grab the muon lifetime data:

In [ ]:
# run this code block without modification
import cloudpickle as cp
from urllib.request import urlopen
lifetime_data = cp.load(urlopen("https://kenelson.web.cern.ch/kenelson/teaching/muon_lifetime.pkl"))

Plot and fit the data.  A suggested additional arugment is `num_bins=40`.

The units on the muon lifetime data are already expressed in microseconds.

So, our distribution doesn't look exactly loke an exponential.  We have too many events with a long lifetime.  To fix this problem, let's restrict ourselves to the data where the measured time was less than 3 microseconds:

In [ ]:
# run this block after you try plotting in the first block
lifetime_data = lifetime_data[lifetime_data<3]
plot_data_and_fit(lifetime_data, function=exponential, num_bins=40)

That looks better!!!

Study how the fit changes with experiment with respect to the number of data points.  To change the number of points you can "slice" the array as `lifetime_data[:N]` where N is the number of data points.  A suggested progression is a set of logarithmically spaced points from 100 to 60000.  You can also try to make a similar plot to the one you made above for the probability changing as a function of the number of data points. This time, see how the best fit value of `tau` changes as you increase the statistics of the data sample.  You can access the best fit value of tau as `yourDict["tau"][0]`.

In [ ]:
sizes = [int(f) for f in np.logspace(2,5,num=20)]
exp_probs = []

for s in sizes:
    local_data = lifetime_data[:s]
    # write a body for this loop which appends to the list
    
# make a plot with sizes on the x axis and exp_probs on the y axis

Try plotting the error on tau: `yourDict["tau"][1]`.  How does the best fit error change as we collect more data?

In [2]:
# copy and paste the above code block but change ["tau"][0] to ["tau"][1]

Next, use `plt.errorbar` to make a plot where the x axis is again the number of observations and on the y axis is the best fit muon lifetime (accessed via `dict["tau"][0]`) and the error bars are the error estimate (accessed via `dict["tau"][0]`).

In [ ]:
sizes = [int(f) for f in np.logspace(2,5,num=20)]
fit_tau = []
fit_err = []

for s in sizes:
    local_data = lifetime_data[:s]
    # save the dictionary and append to the tau and error lists
    
# make a plot with error bars

### Challenge Exercise: "Bump Hunt"

In particle physics one way new particles are discovered is via a bump hunt: a smooth "background" spectrum is fit with an exponential or other similar function and a small signal bump appears at a specific mass point.  For example, this is a plot of the mass of two photons coming from a Higgs boson decay in the ATLAS detector:

<div>
<img src="higgs.jpg" width="500"/>
</div>

In this exercise we will investigate toy data and see if we can find a small bump and estimate the probability that this bump is a statistical fluctuation.

Note that we are still using an exponential function but now the x axis is the invariant mass of a pair of photons produced at the LHC.  I have generated mystery data that you will download:

In [ ]:
# run this code block without modification
mystery_data_small = cp.load(urlopen("https://kenelson.web.cern.ch/kenelson/teaching/mystery_data_1.pkl"))

First, use `plot_data_and_fit` and see if the observed data is compatible with an exponential fit

The data does look like an exponetial with the parameter `tau` near 1.

Now, run the next block of code to load the next data set.  They were generated with the same distribution but the second has more statistics.

Then, plot and fit this data to the `exponential` function.

In [ ]:
# run this code block without modification
mystery_data = cp.load(urlopen("https://kenelson.web.cern.ch/kenelson/teaching/mystery_data_2.pkl"))

It doesn't look so exponential any more!!! There is a clear "bump" between 1 and 2.

Plot the value of the chi-square probability as a function of the size of the dataset.  You can get a subset of the data inside a `for` loop by executing the line `local_data = mystery_data[:s]` where `s` is some `int` telling you how many data entries to use.  The code to initialize the different sizes of each slice is already written for you.

When you call the function `plot_data_and_fit` inside the `for` loop you should add the argument `doPlot=False` to not make a plot for every fit.  Then, you can store the output dictionary and access the probability:
```
exp_dict = plot_data_and_fit(...)
exp_probs.append(exp_dict["prob"])
```

In [ ]:
sizes = [int(f) for f in np.logspace(1.5,np.log10(len(mystery_data)),num=20)]
exp_probs = []

# write the for loop and plotting code similarly to before

So initially the data looks like an exponential, but as we collect more the probability goes to zero, showing us that the expected model is not compatible with the data!  This is how new particles are discovered!

Next, we need to describe the data with some other probability distribution.  It can be a combination of an exponential and gaussian.  The function to define this probability distribution should have five parameters: `x, tau, mu, sigma, gaus_frac` where `gaus_frac` is the fraction of the total probability which is weighted to the gaussian.  Plot the `mystery_data` with an `exponential` fit and an `exponential_plus_gaussian` fit and compare them.

Hint: you can use the existing functions `exponential` and `gaussian` to create your new function `exponential_plus_gaussian`.  The format could look something like:

```
def exponential_plus_gaussian(...):
    return (1-gaus_frac)*exponential(...) + gaus_frac*gaussian(...)
```

When you fit with the `exponential_plus_gaussian` you will need to supply a new argument `guess=[...]` to `plot_data_and_fit`.  This is the best guess initial value of the 4 parameters.  Estimate them by eye from the plot.

Compare the probabilities of the two fits as a function of the number of observations.  This should look very similar to your code from exercise 5: goodness of fit.

Another goodness of fit statistic is the value `chi-sq`/`NDF`, the value of the chi-square test statistic over the number of degrees of freedom.  Plot this as a function of the data points (the relevant information is stored in the dictionary returned by `plot_data_and_fit`).  Does it change much?  Why or why not?

If you are finished, continue to play with the data and plot other values with respect to N observations.  You can also try reading the code in `plot_data_and_fit` and try to digest what is going on.